# Preprocessing of updated OTU data from SILVA / QIIME 2
- Data from Cliff, used QIIME 2

**Steps are:**


**1)** Reformatting taxonomy to match strings in old iTagger output
-  `reformed_Silva_OTU_table.txt`

**2)** Preprocessing: sort samples by west to east, min samples total >= 500/168 samples
- `Silva_OTU_PP.txt`

**3)** DESeq normalization: Variance Stablized Counts Per Million 
- `Silva_OTU_VSTcpm.txt`

# 1)  Get & reformat OTU table

### a) load as is from rds (contains taxa, OTUs, metadata)

In [1]:
data <- readRDS("input_filt.rds")
#data # slow 

In [2]:
# split data
taxa = data$taxonomy
OTUs = data$data_loaded
# metadata = data$map_loaded
# head(OTUs)

In [3]:
# note taxa do not match old format, around which all downstream processing is built! 
# head(taxa)

In [4]:
# Gather raw OTU table - had checked alignment
raw_OTU_table = data.frame(OTUs, taxa)  # note here that asv ordering is preserved -- will use it!
# head(raw_OTU_table)#; tail(raw_OTU_table)  # also note, split taxa not format I want! 

write.table(raw_OTU_table, "Silva_OTU_raw.txt", sep = '\t')

### b)  reformat taxonomy 
- to match older OTU table version with "Consensus.lineage"
- So can consistently use 1_OTU_preprocess_module & label terminal confident taxonomic assignment
- This was initially a single string "Consensus.lineage"
- ranks were ";" separated, led with "k__", "p__", "c__", "o__", "f__", "g__"  
- does not appear that there were species?  or these were just OTUs.
- NO SPECIES INCLUDED HERE!  will need to worry about ASV IDs for later?

In [5]:
# old format had no NAs
taxa[taxa=='NA'] <-""   # no NAs
# head(taxa) 

In [6]:
# make small data for test
sm_taxa = head(taxa)

# Add back in tax rank markers "k__", "p__", "c__", "o__", "f__", "g__"

paste_ranks = function(sm_taxa){
    k = data.frame(k ="k__", sm_taxa['taxonomy1'])
    k2 <- do.call(paste, c(k, sep = ""))

    p = data.frame(k ="p__", sm_taxa['taxonomy2'])
    p2 <- do.call(paste, c(p, sep = ""))

    c = data.frame(k ="c__", sm_taxa['taxonomy3'])
    c2 <- do.call(paste, c(c, sep = ""))

    o = data.frame(k ="o__", sm_taxa['taxonomy4'])
    o2 <- do.call(paste, c(o, sep = ""))

    f = data.frame(k ="f__", sm_taxa['taxonomy5'])
    f2 <- do.call(paste, c(f, sep = ""))

    g = data.frame(k ="g__", sm_taxa['taxonomy6'])
    g2 <- do.call(paste, c(g, sep = ""))

    # NOT USING SPECIES HERE! OTU preprocessing doesn't!
    s = data.frame(k ="s__", sm_taxa['taxonomy7'])
    s2 <- do.call(paste, c(s, sep = ""))

    # combine all
    lineage_df = data.frame(k2, p2, c2, o2, f2, g2)
    lineage = do.call(paste, c(lineage_df, sep = ';'))
    return(lineage)
}

In [7]:
# check this maps for small taxa test
sm_lineage = paste_ranks(sm_taxa)
# sm_taxa['Consensus.lineage'] = sm_lineage
# sm_taxa

In [8]:
# make a small OTU table for testing next steps
sm_otu = head(OTUs)
Consensus.lineage = sm_lineage

sm_OTU_table = data.frame(sm_otu, Consensus.lineage)  # note here that asv ordering is preserved -- will use it!
# head(sm_OTU_table)#; tail(raw_OTU_table)  # also note, split taxa not format I want! 

write.table(sm_OTU_table, "sm_reformed_Silva_OTU_table.txt", sep = '\t')

In [9]:
# Make & export the real thing
Consensus.lineage = paste_ranks(taxa)
reformed_OTU_table = data.frame(OTUs, Consensus.lineage)  # note here that asv ordering is preserved -- will use it!

write.table(reformed_OTU_table, "reformed_Silva_OTU_table.txt", sep = '\t')

# 2) OTU table preprocessing 

In [10]:
#source("../modules/1_OTU_preprocess_module_0.2.r")
source("../modules/1_OTU_preprocessing.r")

### small test shows a problem with sample ordering: 
- Sandmound_Cattail sites were supposed to be dropped (only 2 cores, not v. important habitat) 
- Still there, and now "Sandmound_Tule_C_D1	Sandmound_Tule_C_D2" at the end after Consensus.lineage()
- Above samples have an extra dash before "C_D1, C_D2), fix it.

In [62]:
### IMPORT Sample mapping
metaDB <-read.table("../data/meta/SF_sal_meta_FIX3.txt", sep="\t", header=TRUE)               # import Mapping    # # try keeping all params...
row.names(metaDB) <- metaDB$Sample                                                            # Row names are samples for phyloseq             #head(map_iTag)
metaDB = metaDB[,-1]                                                                          # Drop only old index, keep everything else            
# colnames(metaDB)

## IMPORT OTU TABLE          
# sm_test done first
# otu_raw <- reformed_OTU_table
otu_raw <- read.table("reformed_Silva_OTU_table.txt", sep='\t', header=TRUE)      # add to fxn below?
#otu_tax <-data.frame(OTU= row.names(otu_raw), Taxonomy = otu_raw$Consensus.lineage)      

# Fix sample names to match metadata !!   - discovered with small data test
oldnames = c('Sandmound_Tule_C_D1','Sandmound_Tule_C_D2')
newnames = c('Sandmound_TuleC_D1','Sandmound_TuleC_D2')
for(i in 1:2) names(otu_raw)[names(otu_raw) == oldnames[i]] = newnames[i]

# PREPROCESS OTU TABLE (site sort, filter, taxonomy)
otu_PP = otu_t_preproc(otu_raw, 500, metaDB, "Sample", "EWsiteHyd_index")
otu_PP = otu_t_preproc(otu_raw, 50, metaDB, "Sample", "EWsiteHyd_index")
# dim(otu_PP); head(otu_PP) # names(otu_PP)

# for unknown reasons, need to make OTU counts NUMERIC
exclude <- c("OTU", 'Consensus.lineage', 'Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus')
all_cols <-colnames(otu_PP)
sites = all_cols[all_cols != exclude]
otu_PP[sites] <- lapply(otu_PP[sites], as.numeric)


In [47]:
#head(otu_PP)

In [63]:
# write table
#write.table(otu_PP, "Silva_OTU_PP.txt", sep='\t')
write.table(otu_PP, "Silva_OTU_PP_50.txt", sep='\t')

# 3) DESeq2 normalize data
- VST_CPM here is variance stablized transform, returned as Counts Per Million

In [54]:
source("../modules/2_OTU_table_to_DESeq2_and_VST_cpm_0.7.r")

In [66]:
# Reimport from file!   NOTE TWO VERSIONS, one with 500 and one with 50 cutoff
otu_PP = read.table("Silva_OTU_PP.txt", sep='\t')
otu_PP50 = read.table("Silva_OTU_PP_50.txt", sep='\t')


### IMPORT Sample mapping
metaDB <-read.table("../data/meta/SF_sal_meta_FIX3.txt", sep="\t", header=TRUE)               # import Mapping    # # try keeping all params...
row.names(metaDB) <- metaDB$Sample                                                            # Row names are samples for phyloseq             #head(map_iTag)
metaDB = metaDB[,-1]                                                                          # Drop only old index, keep everything else            

In [67]:
# oversight in the module, looks like fxn 4 needs to take physeq, pass to fxn 5 too.
# just add to global env here, sigh.  or fix in v0.7 new vs v0.6
# physeq = make_Phyloseq_data(otu_PP, metaDB)  

### Get DESeq2 normalized counts per million ###
OTU_vstCPM = calc_DESeq2_CPM(otu_PP50, metaDB)

Warning message in .local(object):
“Coercing from data.frame class to character matrix 
prior to building taxonomyTable. 
This could introduce artifacts. 
Check your taxonomyTable, or coerce to matrix manually.”


phyloseq-class experiment-level object
otu_table()   OTU Table:         [ 43320 taxa and 168 samples ]
sample_data() Sample Data:       [ 168 samples by 66 sample variables ]
tax_table()   Taxonomy Table:    [ 43320 taxa by 8 taxonomic ranks ]


converting counts to integer mode

Warning message in DESeq(OTU_phy2des, test = "Wald", fitType = "parametric"):
“the design is ~ 1 (just an intercept). is this intended?”
estimating size factors

estimating dispersions

gene-wise dispersion estimates

mean-dispersion relationship

-- note: fitType='parametric', but the dispersion trend was not well captured by the
   function: y = a/x + b, and a local regression fit was automatically substituted.
   specify fitType='local' or 'mean' to avoid this message next time.

final dispersion estimates

fitting model and testing

-- replacing outliers and refitting for 10934 genes
-- DESeq argument 'minReplicatesForReplace' = 7 
-- original counts are preserved in counts(dds)

estimating dispersions

fitting model and testing



[1] "Deseq2 finished computing"


In [68]:
head(OTU_vstCPM)

,OTU,Sandmound_TuleA_D1,Sandmound_TuleA_D2,Sandmound_TuleB_D1,Sandmound_TuleB_D2,Sandmound_TuleC_D1,Sandmound_TuleC_D2,Sandmound_CattailA_D1,Sandmound_CattailA_D2,Sandmound_ThreeSqA_D1,⋯,Muzzi_PWB_D2,Muzzi_PWC_D1,Muzzi_PWC_D2,Consensus.lineage,Kingdom,Phylum,Class,Order,Family,Genus
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>
1,otu_ASV_1,369.8468,490.46468,197.17816,765.73070,271.38947,759.66376,593.57706,468.23556,583.93556,⋯,296.20853,1018.07076,792.189328,k__Bacteria;p__Firmicutes;c__Bacilli;o__Bacillales;f__;g__,Bacteria,Firmicutes,Bacilli,Bacillales,BacillalesOR,BacillalesOR
2,otu_ASV_10,183.1951,387.00728,481.99106,132.89541,14.03739,263.40100,387.27284,11.06068,20.70693,⋯,256.71406,1678.73369,592.830424,k__Bacteria;p__Desulfobacterota;c__Desulfuromonadia;o__Desulfuromonadales;f__Sva1033;g__,Bacteria,Desulfobacterota,Desulfuromonadia,Desulfuromonadales,Sva1033,Sva1033FA
3,otu_ASV_100,255.7819,91.96213,109.54342,37.97012,42.11216,19.08703,361.93723,114.29372,24.84832,⋯,19.74724,16.24581,15.738861,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Arenicellales;f__Arenicellaceae;g__,Bacteria,Proteobacteria,Gammaproteobacteria,Arenicellales,Arenicellaceae,ArenicellaceaeFA
4,otu_ASV_1000,407.8684,341.02622,21.90868,259.46247,575.53284,488.62795,351.07912,652.58027,120.10022,⋯,19.74724,5.41527,5.246287,k__Bacteria;p__Myxococcota;c__Polyangia;o__Polyangiales;f__BIrii41;g__,Bacteria,Myxococcota,Polyangia,Polyangiales,BIrii41,BIrii41FA
5,otu_ASV_10000,58.7607,19.15878,21.90868,69.61188,70.18693,19.08703,94.10368,29.49515,103.53467,⋯,19.74724,5.41527,5.246287,k__Bacteria;p__Proteobacteria;c__Gammaproteobacteria;o__Burkholderiales;f__Comamonadaceae;g__Aquabacterium,Bacteria,Proteobacteria,Gammaproteobacteria,Burkholderiales,Comamonadaceae,Aquabacterium
6,otu_ASV_10001,197.0212,42.14931,21.90868,18.98506,201.20254,72.53071,47.05184,18.43447,194.64519,⋯,19.74724,5.41527,5.246287,k__Bacteria;p__Verrucomicrobiota;c__Verrucomicrobiae;o__Pedosphaerales;f__Pedosphaeraceae;g__,Bacteria,Verrucomicrobiota,Verrucomicrobiae,Pedosphaerales,Pedosphaeraceae,PedosphaeraceaeFA


In [69]:
# export it
write.table(OTU_vstCPM, "Silva_OTU_VSTcpm50.txt", sep='\t')